<a href="https://colab.research.google.com/github/TheAmirHK/BirdMurmuration/blob/main/BirdMurmuration_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Created on Fri Dec 13 12:00:40 2024

@author: amirh
"""
import os
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from gymnasium import spaces
from stable_baselines3 import PPO
import imageio

In [ ]:
# In[Enviroenment]
class MurmurationEnv(gym.Env):
    def __init__(self, num_birds=5):
        super(MurmurationEnv, self).__init__()
        self.num_birds = num_birds
        self.observation_space = spaces.Box(low=-1, high=1, shape=(num_birds * 6,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1, high=1, shape=(num_birds * 3,), dtype=np.float32)
        self.frames = []
        self.frame_dir = "frames"
        os.makedirs(self.frame_dir, exist_ok=True)
        self.frame_count = 0
        self.reset()
        self.steps = 0
        self.max_steps = 1000

    def reset(self, seed=None, options=None):
        self.positions = np.random.uniform(-1, 1, (self.num_birds, 3))
        self.velocities = np.random.uniform(-0.1, 0.1, (self.num_birds, 3))
        self.steps = 0
        return self._get_obs(), {}

# flocking behavior avoids collisions, maintain neighbors’ velocities and moves toward the center.
    def _flocking_rules(self):
        for i in range(self.num_birds):
            neighbors = [j for j in range(self.num_birds) if j != i]
            if not neighbors:
                continue

            # avoids collisions
            separation = np.sum([self.positions[i] - self.positions[j] for j in neighbors if np.linalg.norm(self.positions[i] - self.positions[j]) < 0.2], axis=0, initial=0)

            # maintain neighbors’ velocities
            alignment = np.mean([self.velocities[j] for j in neighbors], axis=0) - self.velocities[i]

            # moves toward the center
            center_of_mass = np.mean([self.positions[j] for j in neighbors], axis=0)
            cohesion = center_of_mass - self.positions[i]


            self.velocities[i] += 0.01 * (separation + alignment + cohesion)

    def _calculate_reward(self):
        alignment_reward = np.mean(np.linalg.norm(np.mean(self.velocities, axis=0) - self.velocities, axis=1))
        cohesion_reward = -np.mean(np.linalg.norm(self.positions - np.mean(self.positions, axis=0), axis=1))
        return alignment_reward + cohesion_reward

    def step(self, action):
        self.steps += 1

        action = action.reshape((self.num_birds, 3))
        self.velocities += action * 0.1  # velocity
        self.velocities = np.clip(self.velocities, -0.1, 0.1)
        self.positions += self.velocities  # change position

        self._flocking_rules()

        # reward system
        reward = self._calculate_reward()
        done = bool(self.steps >= self.max_steps)
        print(self.steps)
        return self._get_obs(), reward, done, False, {}

    def _get_obs(self):
        return np.concatenate([self.positions.flatten(), self.velocities.flatten()], dtype=np.float32)

    def render(self):
        fig, axes = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(10, 5))
        rotations = [(15, 15), (45, 45)]

        for ax, (elev, azim) in zip(axes, rotations):
            ax.scatter(self.positions[:, 0], self.positions[:, 1], self.positions[:, 2], c='black')
            ax.set_xlim(-3, 3)
            ax.set_ylim(-3, 3)
            ax.set_zlim(-3, 3)
            ax.view_init(elev=elev, azim=azim)

        frame_path = os.path.join(self.frame_dir, f"frame_{self.frame_count:04d}.png")
        plt.savefig(frame_path)
        self.frame_count += 1
        plt.close(fig)

    def save_animation(self, filename="murmuration.gif"):
        images = []
        for i in range(self.frame_count):
            img_path = os.path.join(self.frame_dir, f"frame_{i:04d}.png")
            images.append(imageio.imread(img_path))
        imageio.mimsave(filename, images, fps=10)

In [ ]:
# In[Train]
env = MurmurationEnv(num_birds=200)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=5000)

In [ ]:
# In[Test]
obs, _ = env.reset()
print("Test started ...")
for _ in range(500):
    action, _ = model.predict(obs)
    obs, _, done, _, _ = env.step(action)
    env.render()
    if done:
        break
env.close()

In [ ]:
# In[Save animation]
env.save_animation("murmuration.gif")